<h1>Analysis of Global COVID-19 Pandemic Data</h1>

Estimated time needed: **90** minutes



## Overview:

There are 10 tasks in this final project. All tasks will be graded by your peers who are also completing this assignment within the same session.

You need to submit the following the screenshot for the code and output for each task for review.

If you need to refresh your memories about specific coding details, you may refer to previous hands-on labs for code examples.


In [97]:
# This lab requires 'httr' and 'rvest'packages, which are already pre-loaded into this lab environment.
# However, if you are working on your local RStudio, please uncomment the below codes and install the packages.

#install.packages("httr")
#install.packages("rvest")

In [98]:
library(httr)
library(rvest)

Note: if you can import above libraries, please use install.packages() to install them first.


## TASK 1: Get a `COVID-19 pandemic` Wiki page using HTTP request


First, let's write a function to use HTTP request to get a public COVID-19 Wiki page.

Before you write the function, you can open this public page from this

URL https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country using a web browser.

The goal of task 1 is to get the html page using HTTP request (`httr` library)


In [99]:
get_wiki_covid19_page <- function() {

  # Our target COVID-19 wiki page URL is: https://en.wikipedia.org/w/index.php?title=Template:COVID-19_testing_by_country
  # Which has two parts:
    # 1) base URL `https://en.wikipedia.org/w/index.php
    # 2) URL parameter: `title=Template:COVID-19_testing_by_country`, seperated by question mark ?

  # Wiki page base
  wiki_base_url <- "https://en.wikipedia.org/w/index.php"
  # You will need to create a List which has an element called `title` to specify which page you want to get from Wiki
  query_params <- list(title = "Template:COVID-19_testing_by_country")
  # in our case, it will be `Template:COVID-19_testing_by_country`

  # - Use the `GET` function in httr library with a `url` argument and a `query` arugment to get a HTTP response
  response <- GET(url = wiki_base_url, query = query_params)

  # Use the `return` function to return the response
  return(response)

}

Call the `get_wiki_covid19_page` function to get a http response with the target html page


In [100]:
# Call the get_wiki_covid19_page function and print the response
covid_response <- get_wiki_covid19_page()
print(covid_response)



Response [https://en.wikipedia.org/w/index.php?title=Template%3ACOVID-19_testing_by_country]
  Date: 2026-01-04 14:32
  Status: 200
  Content-Type: text/html; charset=UTF-8
  Size: 457 kB
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-fea...
<head>
<meta charset="UTF-8">
<title>Template:COVID-19 testing by country - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-heade...
RLSTATE={"ext.globalCssJs.user.styles":"ready","site.styles":"ready","user.st...
<script>(RLQ=window.RLQ||[]).push(function(){mw.loader.impl(function(){return...
}];});});</script>
<link rel="stylesheet" href="/w/load.php?lang=en&amp;modules=ext.cite.styles%...
...


## TASK 2: Extract COVID-19 testing data table from the wiki HTML page


On the COVID-19 testing wiki page, you should see a data table `<table>` node contains COVID-19 testing data by country on the page:

<a href="https://cognitiveclass.ai/">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/M5_Final/images/covid-19-by-country.png" width="400" align="center">
</a>

Note the numbers you actually see on your page may be different from above because it is still an on-going pandemic when creating this notebook.

The goal of task 2 is to extract above data table and convert it into a data frame


Now use the `read_html` function in rvest library to get the root html node from response


In [101]:
# Get the root html node from the http response in task 1
root_node <- read_html(covid_response)

Get the tables in the HTML root node using `html_nodes` function.


In [102]:
# Get the table node from the root html node
table_node <- html_nodes(root_node, "table")

Read the specific table from the multiple tables in the `table_node` using the `html_table` function and convert it into dataframe using `as.data.frame`

_Hint:- Please read the `table_node` with index 2(ex:- table_node[2])._


In [103]:
# Read the table node and convert it into a data frame, and print the data frame for review
covid_data_frame <- html_table(table_node[2], fill = TRUE)[[1]]

## TASK 3: Pre-process and export the extracted data frame

The goal of task 3 is to pre-process the extracted data frame from the previous step, and export it as a csv file


Let's get a summary of the data frame


In [104]:
# Print the summary of the data frame
head(covid_data_frame)
print(dim(covid_data_frame))

Country or region,Date[a],Tested,Units[b],Confirmed(cases),"Confirmed /tested,%","Tested /population,%","Confirmed /population,%",Ref.
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Afghanistan,17 Dec 2020,"154,767",samples,"49,621",32.1,0.40,0.13,[1]
Albania,18 Feb 2021,"428,654",samples,"96,838",22.6,15.0,3.4,[2]
Algeria,2 Nov 2020,"230,553",samples,"58,574",25.4,0.53,0.13,[3][4]
Andorra,23 Feb 2022,"300,307",samples,"37,958",12.6,387,49.0,[5]
Angola,2 Feb 2021,"399,228",samples,"20,981",5.3,1.3,0.067,[6]
Antigua and Barbuda,6 Mar 2021,"15,268",samples,832,5.4,15.9,0.86,[7]


[1] 173   9


As you can see from the summary, the columns names are little bit different to understand and some column data types are not correct. For example, the `Tested` column shows as `character`.

As such, the data frame read from HTML table will need some pre-processing such as removing irrelvant columns, renaming columns, and convert columns into proper data types.


We have prepared a pre-processing function for you to conver the data frame but you can also try to write one by yourself


In [105]:
preprocess_covid_data_frame <- function(data_frame) {

    shape <- dim(data_frame)

    # Remove the World row
    data_frame<-data_frame[!(data_frame$`Country or region`=="World"),]
    # Remove the last row dynamically
    data_frame <- data_frame[1:(nrow(data_frame) - 1), ]

    # We dont need the Units and Ref columns, so can be removed
    data_frame["Ref."] <- NULL
    data_frame["Units[b]."] <- NULL

    # Renaming the columns
    names(data_frame) <- c("country", "date", "tested", "confirmed", "confirmed.tested.ratio", "tested.population.ratio", "confirmed.population.ratio")

    # Convert column data types
    data_frame$country <- as.factor(data_frame$country)
    data_frame$date <- as.factor(data_frame$date)
    data_frame$tested <- as.numeric(gsub(",","",data_frame$tested))
    data_frame$confirmed <- as.numeric(gsub(",","",data_frame$confirmed))
    data_frame$'confirmed.tested.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.tested.ratio`))
    data_frame$'tested.population.ratio' <- as.numeric(gsub(",","",data_frame$`tested.population.ratio`))
    data_frame$'confirmed.population.ratio' <- as.numeric(gsub(",","",data_frame$`confirmed.population.ratio`))

    return(data_frame)
}

Call the `preprocess_covid_data_frame` function


In [106]:
# call `preprocess_covid_data_frame` function and assign it to a new data frame
covid_data_frame_csv <- preprocess_covid_data_frame(covid_data_frame)

Warning message in preprocess_covid_data_frame(covid_data_frame):
“NAs introduced by coercion”


Get the summary of the processed data frame again


In [107]:
# Print the summary of the processed data frame again
summary(covid_data_frame_csv)

                country             date         tested            confirmed  
 Afghanistan        :  1   2 Feb 2023 :  6   Min.   :     3880   Min.   : NA  
 Albania            :  1   1 Feb 2023 :  4   1st Qu.:   512037   1st Qu.: NA  
 Algeria            :  1   31 Jan 2023:  4   Median :  3029859   Median : NA  
 Andorra            :  1   1 Mar 2021 :  3   Mean   : 31377219   Mean   :NaN  
 Angola             :  1   23 Jul 2021:  3   3rd Qu.: 12386725   3rd Qu.: NA  
 Antigua and Barbuda:  1   29 Jan 2023:  3   Max.   :929349291   Max.   : NA  
 (Other)            :166   (Other)    :149                       NA's   :172  
 confirmed.tested.ratio tested.population.ratio confirmed.population.ratio
 Min.   :       0       Min.   : 0.00           Min.   :   0.0065         
 1st Qu.:   37839       1st Qu.: 5.00           1st Qu.:   9.4750         
 Median :  281196       Median :10.05           Median :  46.9500         
 Mean   : 2508340       Mean   :11.25           Mean   : 175.5043   

After pre-processing, you can see the columns and columns names are simplified, and columns types are converted into correct types.


The data frame has following columns:

- **country** - The name of the country
- **date** - Reported date
- **tested** - Total tested cases by the reported date
- **confirmed** - Total confirmed cases by the reported date
- **confirmed.tested.ratio** - The ratio of confirmed cases to the tested cases
- **tested.population.ratio** - The ratio of tested cases to the population of the country
- **confirmed.population.ratio** - The ratio of confirmed cases to the population of the country


OK, we can call `write.csv()` function to save the csv file into a file.


In [108]:
# Export to CSV
write.csv(covid_data_frame_csv, "covid.csv", row.names = FALSE)
# Export the data frame to a csv file
print(file.exists("covid.csv"))

[1] TRUE


Note for IBM Waston Studio, there is no traditional "hard disk" associated with a R workspace.

Even if you call `write.csv()` method to save the data frame as a csv file, it won't be shown in IBM Cloud Object Storage asset UI automatically.

However, you may still check if the `covid.csv` exists using following code snippet:


In [109]:
# Get working directory
wd <- getwd()
# Get exported
file_path <- paste(wd, sep="", "/covid.csv")
# File path
print(file_path)
file.exists(file_path)

[1] "/content/covid.csv"


[1] TRUE

**Optional Step**: If you have difficulties finishing above webscraping tasks, you may still continue with next tasks by downloading a provided csv file from here:


In [110]:
## Download a sample csv file
# covid_csv_file <- download.file("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0101EN-Coursera/v2/dataset/covid.csv", destfile="covid.csv")
# covid_data_frame_csv <- read.csv("covid.csv", header=TRUE, sep=",")

## TASK 4: Get a subset of the extracted data frame

The goal of task 4 is to get the 5th to 10th rows from the data frame with only `country` and `confirmed` columns selected


In [120]:
# Read covid_data_frame_csv from the csv file
covid_data_frame_csv <- read.csv("covid.csv", header = TRUE, sep = ",")

# Get the 5th to 10th rows, with two "country" "confirmed" columns
subset_data <- covid_data_frame_csv[5:10, c("country", "confirmed")]



## TASK 5: Calculate worldwide COVID testing positive ratio

The goal of task 5 is to get the total confirmed and tested cases worldwide, and try to figure the overall positive ratio using `confirmed cases / tested cases`


In [112]:
# Get the total confirmed cases worldwide
total_confirmed <- sum(covid_data_frame_csv$confirmed, na.rm = TRUE)
print(paste("Total confirmed cases worldwide:", total_confirmed))

# Get the total tested cases worldwide
total_tested <- sum(covid_data_frame_csv$tested, na.rm = TRUE)
print(paste("Total tested cases worldwide:", total_tested))

# Get the positive ratio (confirmed / tested)
positive_ratio <- total_confirmed / total_tested
print(paste("Worldwide positive ratio:", round(positive_ratio, 4)))
print(paste("Worldwide positive percentage:", round(positive_ratio * 100, 2), "%"))


[1] "Total confirmed cases worldwide: 0"
[1] "Total tested cases worldwide: 5396881644"
[1] "Worldwide positive ratio: 0"
[1] "Worldwide positive percentage: 0 %"


## TASK 6: Get a country list which reported their testing data

The goal of task 6 is to get a catalog or sorted list of countries who have reported their COVID-19 testing data


In [113]:
# Get the `country` column
countries <- covid_data_frame_csv$country

# Check its class (should be character)
print(class(countries))

countries_char <- as.character(countries)

# Sort the countries AtoZ
countries_AtoZ <- sort(countries_char)
print("First 10 countries (A to Z):")
print(head(countries_AtoZ, 10))

# Sort the countries ZtoA
countries_ZtoA <- sort(countries_char, decreasing = TRUE)
print("Countries sorted Z to A:")

# Print the sorted ZtoA list
print(countries_ZtoA)

[1] "character"
[1] "First 10 countries (A to Z):"
 [1] "Afghanistan"         "Albania"             "Algeria"            
 [4] "Andorra"             "Angola"              "Antigua and Barbuda"
 [7] "Argentina"           "Armenia"             "Australia"          
[10] "Austria"            
[1] "Countries sorted Z to A:"
  [1] "Zimbabwe"               "Zambia"                 "Vietnam"               
  [4] "Venezuela"              "Uzbekistan"             "Uruguay"               
  [7] "United States"          "United Kingdom"         "United Arab Emirates"  
 [10] "Ukraine"                "Uganda"                 "Turkey"                
 [13] "Tunisia"                "Trinidad and Tobago"    "Togo"                  
 [16] "Thailand"               "Tanzania"               "Taiwan[m]"             
 [19] "Switzerland[l]"         "Sweden"                 "Sudan"                 
 [22] "Sri Lanka"              "Spain"                  "South Sudan"           
 [25] "South Korea"           

## TASK 7: Identify countries names with a specific pattern

The goal of task 7 is using a regular expression to find any countires start with `United`


In [114]:
# Use a regular expression `United.+` to find matches
pattern <- "^United"
matches <- grep(pattern, countries_char, value = TRUE)

# Print the matched country names
print("Countries starting with 'United':")
print(matches)


[1] "Countries starting with 'United':"
[1] "United Arab Emirates" "United Kingdom"       "United States"       


## TASK 8: Pick two countries you are interested, and then review their testing data

The goal of task 8 is to compare the COVID-19 test data between two countires, you will need to select two rows from the dataframe, and select `country`, `confirmed`, `confirmed-population-ratio` columns


In [115]:
# Select a subset (should be only one row) of data frame based on a selected country name and columns

country1 <- "United States"
country1_data <- covid_data_frame_csv[covid_data_frame_csv$country == country1,
                                       c("country", "confirmed", "confirmed.population.ratio")]
print("Country 1 data:")
print(country1_data)

# Select a subset (should be only one row) of data frame based on a selected country name and columns
country2 <- "India"
country2_data <- covid_data_frame_csv[covid_data_frame_csv$country == country2,
                                       c("country", "confirmed", "confirmed.population.ratio")]
print("Country 2 data:")
print(country2_data)

[1] "Country 1 data:"
          country confirmed confirmed.population.ratio
166 United States        NA                        281
[1] "Country 2 data:"
   country confirmed confirmed.population.ratio
73   India        NA                         63


## TASK 9: Compare which one of the selected countries has a larger ratio of confirmed cases to population

The goal of task 9 is to find out which country you have selected before has larger ratio of confirmed cases to population, which may indicate that country has higher COVID-19 infection risk


In [116]:
# Use if-else statement
# if (check which confirmed.population value is greater) {
#    print()
# } else {
#    print()
# }

ratio1 <- country1_data$confirmed.population.ratio
ratio2 <- country2_data$confirmed.population.ratio

if (ratio1 > ratio2) {
  print(paste(country1, "has a higher confirmed-to-population ratio:", ratio1))
  print(paste("This indicates", country1, "has higher COVID-19 infection risk"))
} else if (ratio2 > ratio1) {
  print(paste(country2, "has a higher confirmed-to-population ratio:", ratio2))
  print(paste("This indicates", country2, "has higher COVID-19 infection risk"))
} else {
  print("Both countries have the same confirmed-to-population ratio")
}


[1] "United States has a higher confirmed-to-population ratio: 281"
[1] "This indicates United States has higher COVID-19 infection risk"


## TASK 10: Find countries with confirmed to population ratio rate less than a threshold

The goal of task 10 is to find out which countries have the confirmed to population ratio less than 1%, it may indicate the risk of those countries are relatively low


In [117]:
# Get a subset of any countries with `confirmed.population.ratio` less than the threshold
threshold <- 0.01
low_risk_countries <- covid_data_frame_csv[covid_data_frame_csv$confirmed.population.ratio < threshold, ]

print(paste("Countries with confirmed-to-population ratio less than 1%:"))
print(low_risk_countries[, c("country", "confirmed", "confirmed.population.ratio")])

# Count how many countries
print(paste("Total countries with low risk:", nrow(low_risk_countries)))


[1] "Countries with confirmed-to-population ratio less than 1%:"
     country confirmed confirmed.population.ratio
156 Tanzania        NA                     0.0065
[1] "Total countries with low risk: 1"
